<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_composite_indexes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. https://www.pinecone.io/learn/series/faiss/composite-indexes/
2. https://www.youtube.com/watch?v=GEhmmcx1lvM
3. https://www.youtube.com/watch?v=3Wqh4iUupbM



In [ ]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 0s (1,785 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

首先，我们需要数据。我们将使用[Sift1M 数据集](http://corpus-texmex.irisa.fr/)，我们可以使用以下命令将其下载并加载到笔记本中：


In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [ ]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [ ]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [ ]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

In [ ]:
xq.shape

(1, 128)

In [ ]:
xb.shape

(1000000, 128)

[在向量搜索](https://www.pinecone.io/learn/what-is-similarity-search/)领域，有许多索引方法和向量处理技术，使我们能够在召回率、延迟和内存使用之间确定优先级。

[使用 IVF、 PQ](https://www.pinecone.io/learn/series/faiss/product-quantization/)或[HNSW](https://www.pinecone.io/learn/series/faiss/hnsw/)等特定方法，我们通常可以获得良好的结果。但为了获得*最佳性能，*我们通常希望使用*复合索引*。

我们可以将复合索引视为向量变换和一种或多种索引方法的逐步过程。允许我们将多个索引和/或处理步骤放在一起来创建我们的“理想”索引。

[例如，我们可以使用倒排文件（IVF）索引来缩小搜索范围（提高搜索速度），然后添加乘积量化（PQ）](https://www.pinecone.io/learn/series/faiss/product-quantization/)等压缩技术，以将较大的索引保持在合理的大小限制内。

如果能够自定义索引，则存在生成具有不必要的不良召回、延迟或内存使用率的索引的风险。

如果我们想构建强大且高性能的向量相似性搜索应用程序，我们必须了解复合索引的工作原理。必须了解在哪里可以使用不同的索引或向量转换以及何时不需要它们。

[在本文中，我们将学习如何使用Facebook AI 相似性搜索 (Faiss)](https://www.pinecone.io/learn/series/faiss/)构建高性能复合索引——这是一个强大的库，许多人使用它来构建快速、准确的向量相似性搜索索引。我们还将介绍 Faiss **index_factory**，它允许我们用更清晰、更优雅的代码构建复合索引。

## 什么是复合索引(Composite Indexes)

复合索引类似于*乐高积木*；我们把一个放在另一个上面。我们会发现大多数块都可以组合在一起，但不同的组合可以产生任何东西，从艺术杰作到无法辨认的混乱。

这同样适用于费斯。大多数组件*可以*放置在一起 - 但这并不意味着它们*应该*放置在一起。

综合索引由以下各项的任意组合构建：

- **矢量变换**— 在索引之前应用于矢量的预处理步骤（PCA、OPQ）。
- **粗量化器**-将向量*粗略*组织到子域（用于限制搜索范围，包括 IVF、IMI 和 HNSW）。
- **精细量化器**—将向量*更精细地*压缩为更小的域（用于压缩索引大小，例如 PQ）。
- **细化**——搜索时的最后一步，使用原始平面向量上的距离计算对结果进行重新排序。或者，可以使用另一个索引（非平坦）索引。

请注意，粗量化是指向量的“聚类”（例如 IVF 的反向索引）。通过使用粗量化，我们通过限制搜索范围来实现*非穷举搜索。*

精细量化描述了将向量压缩为*代码*（与 PQ 一样）[1][2][3]。这样做的目的是为了减少索引的内存使用。

### 指数成分

我们可以使用以下组件构建复合索引：

|            矢量变换            |                           粗量化器                           |                  精细量化器                   |     细化      |
| :----------------------------: | :----------------------------------------------------------: | :-------------------------------------------: | :-----------: |
| PCA、OPQ、RR、L2norm、ITQ、Pad | IVF、扁平、IMI、IVF、HNSW、IVF、PQ、IVF、RCQ、HNSW、扁平、HNSW、SQ、HNSW、PQ | 扁平*、PQ、SQ、残差*、RQ、LSQ、ZnLattice、LSH | RF扁平、精炼* |

例如，我们可以构建一个索引，其中：

- **使用OPQ**转换传入向量。
- 通过将向量存储在倒排文件列表**IVF**中来执行向量的粗量化，从而实现非穷举搜索。
- 压缩向量，通过每个 IVF 单元内的**PQ**减少内存使用*（向量被量化，但它们的单元分配不会改变）*。
- 搜索后，根据原始平面向量**RFlat**对结果重新排序。

**构建这些索引时，使用不同 Faiss 类的列表可能会变得混乱 - 因此使用 Faiss index_factory**构建索引通常会更清晰。

![我们可以合并 IVF 和 PQ 索引，将量化的 PQ 向量存储在 IVF 结构中。](https://cdn.sanity.io/images/vr8gru94/production/409e12bad9ca16b82d685bf6c203ea9c0d282b59-1920x1080.png)

我们可以合并 IVF 和 PQ 索引，将量化的 PQ 向量存储在 IVF 结构中。

## Faiss Index Factory

Faiss **index_factory**函数允许我们使用字符串来构建复合索引。它允许我们切换：

In [30]:
import faiss
quantizer = faiss.IndexFlatL2(128)
index = faiss.IndexIVFFlat(quantizer, 128, 256)
#index.is_trained
index.train(xb)
index.add(xb)

In [33]:
index_f = faiss.index_factory(128, "IVF256,Flat")
#index_f.is_trained
index_f.train(xb)
index_f.add(xb)


我们没有在 index_factory 示例中指定L2距离，因为 index_factory 默认使用L2。如果我们想使用 IndexFlatIP， 我们将 faiss.METRIC_INNER_PRODUCT 添加到我们的 index_factory 参数中。

通过比较它们的性能，我们可以确认这两种方法产生相同的综合指数。首先，它们是否返回相同的最近邻居？



In [34]:
k = 100
D, I = index.search(xq, k)  # search class-based index
print(I,D)

[[701258 455537 562594 600499 619660  36267   2176 454263 722642 779712
  721706 480497 619829 701919 224263 871568 225116 904911 391655 957845
  930567 882961 221339 486457 104122 710644 465294 919197  16429 398306
  989762 670103 403035 656997  87759 293551 839679 679408 931855 169794
  703365 722837 882943  14218 392032  95545 932068 828457 782458 436743
  403442 565750 977462 480765 875747 703631 561735 564474 185891 491343
  454389 831924 727687 253993 827372 377461 629490 245147 746238 574941
  955881 222238 730997 722935 744176 170348 839745 394538 879897  88584
  628092 174301 172378 753423 310902 931722 842594 873615 820116  20822
  742261  15892 236119 710839 656501 312288 562343 561910 928461 710969]] [[69844. 71441. 73537. 73793. 74356. 76583. 76608. 76664. 77587. 78092.
  78655. 79223. 79309. 79508. 80246. 80499. 80639. 80668. 80749. 81399.
  81870. 81991. 82049. 82210. 82455. 82615. 82742. 83031. 83307. 83377.
  83466. 83576. 83632. 83646. 83828. 83899. 83911. 84504. 8461

In [35]:
D_f, I_f = index_f.search(xq, k)  # search index_factory index
print(I_f,D_f)

[[701258 455537 562594 600499 619660  36267   2176 454263 722642 779712
  721706 480497 619829 701919 224263 871568 225116 904911 391655 957845
  930567 882961 221339 486457 104122 710644 465294 919197  16429 398306
  989762 670103 403035 656997  87759 293551 839679 679408 931855 169794
  703365 722837 882943  14218 392032  95545 932068 828457 782458 436743
  403442 565750 977462 480765 875747 703631 561735 564474 185891 491343
  454389 831924 727687 253993 827372 377461 629490 245147 746238 574941
  955881 222238 730997 722935 744176 170348 839745 394538 879897  88584
  628092 174301 172378 753423 310902 931722 842594 873615 820116  20822
  742261  15892 236119 710839 656501 312288 562343 561910 928461 710969]] [[69844. 71441. 73537. 73793. 74356. 76583. 76608. 76664. 77587. 78092.
  78655. 79223. 79309. 79508. 80246. 80499. 80639. 80668. 80749. 81399.
  81870. 81991. 82049. 82210. 82455. 82615. 82742. 83031. 83307. 83377.
  83466. 83576. 83632. 83646. 83828. 83899. 83911. 84504. 8461

In [36]:
I_f.tolist() == I.tolist()  # check that both output same results

True

相同的结果，它们的搜索速度和内存使用情况如何比较？



In [37]:
%%timeit
index.search(xq, k)

116 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [38]:
%%timeit
index_f.search(xq, k)

117 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
import os

def get_memory(index):
    # write index to file
    faiss.write_index(index, './temp.index')
    # get file size
    file_size = os.path.getsize('./temp.index')
    # delete saved index
    os.remove('./temp.index')
    return file_size

In [40]:
get_memory(index)

520133259

In [41]:
get_memory(index_f)

520133259

get_memory函数返回内存使用情况的精确匹配。搜索速度非常接近，index_factory版本快了 5μs——差异可以忽略不计。

 我们将召回率计算为平坦 L2 索引与测试索引之间top- k的匹配百分比。

文献中更常用的指标是recall@k；这不是 这里 计算的召回率。 Recall@k 是返回前k 个 返回记录中最近邻居的查询的百分比。

 如果我们在使用k 值为 100时有 50% 的时间返回真实的最近邻，则我们可以说recall@100 性能为 0.5。



## 为什么使用索引工厂
从我们的测试来看，我们可以确信这两种索引构建方法只不过是通往同一目的地的不同路径。

考虑到这一点——我们为什么要关心学习如何使用index_factory？首先，这可以取决于个人喜好。如果您更喜欢基于类的索引构建方法，请坚持使用。

然而，通过使用index_factory我们可以大大提高代码的优雅性和清晰度。当使用index_factory时，我们将看到五行复杂的代码可以用一行更易读的代码表示。

让我们组合一个复合索引，其中使用 OPQ 预处理向量，使用 IVF 聚类，使用 PQ 量化，然后使用平面索引重新排序。



In [ ]:
d = xb.shape[1]
m = 32
nbits = 8
nlist = 256

# we initialize our OPQ and coarse+fine quantizer steps separately
opq = faiss.OPQMatrix(d, m)
# d now refers to shape of rotated vectors from OPQ (which are equal)
vecs = faiss.IndexFlatL2(d)
sub_index = faiss.IndexIVFPQ(vecs, d, nlist, m, nbits)
# now we merge the preprocessing, coarse, and fine quantization steps
index = faiss.IndexPreTransform(opq, sub_index)
# we will add all of the previous steps to our final refinement step
index = faiss.IndexRefineFlat(q)
# train the index, and index vectors
index.train(xb)
index.add(xb)

此代码演示了向索引添加多个组件可能产生的复杂性。如果我们使用index_factory重写它，我们会得到更简单的代码：


In [ ]:
d = xb.shape[1]
# in index string, m==32, nlist==256, nbits is 8 by default

index = faiss.index_factory(d, "OPQ32,IVF256,PQ32,RFlat")
# train and index vectors
index.train(xb)
index.add(xb)

两种方法产生完全相同的索引。每个的性能：

|              | 记起 | 搜索时间 | 内存使用情况 |
| :----------: | :--: | :------: | :----------: |
| 没有索引工厂 | 31%  | 181微秒  |    552MB     |
| 使用索引工厂 | 31%  | 174微秒  |    552MB     |

**使用index_factory**时，搜索时间确实会稍微快一些，但除此之外，使用或不使用**index_factory**构建的等效索引之间没有性能差异。

## 热门复合索引

现在我们知道如何使用**index_factory**快速构建复合索引，让我们探索一些流行的高性能组合。

## IVFADC

我们在上面介绍了修改后的**IVFADC**索引 -我们之前示例中的**IVF256、PQ32**部分构成了 IVFADC 的核心。让我们更详细地探讨一下。

该指数于 2010 年与产品量化一起引入[4]。从那时起，它仍然是最受欢迎的索引之一 - 由于它是一种易于使用的索引，可以产生合理的召回率、快速的速度和*令人难以置信的*内存使用率。

当我们的首要任务是最大限度地减少内存使用同时保持快速搜索速度时，IVFADC 是理想的选择。*这是以好的*但不好*的*召回性能为代价的。

使用 IVFADC 建立索引有两个步骤：

1. 向量被分配到IVF 结构中的不同列表（*或Voronoi 单元）。*
2. 使用 PQ 压缩向量。

![IVFADC 的索引过程，改编自[4]。](https://cdn.sanity.io/images/vr8gru94/production/0b9d996a8476e1bbe0ceea54ea7703f304125ffd-1920x980.png)

IVFADC 的索引过程，改编自[4]。

对向量进行索引后，在查询向量**xq**和我们的索引量化向量**之间**执行对称距离计算 ( **ADC** ) **。**

该搜索被称为*非对称搜索*，因为它将未压缩的**xq与压缩的 PQ 向量（我们之前索引的）进行比较。**

![通过对称距离计算（SDC，左），我们在将 xq 与之前量化的 xb 向量进行比较之前对其进行量化。 ADC（右）跳过 xq 的量化，并将其直接与量化的 xb 向量进行比较。](https://cdn.sanity.io/images/vr8gru94/production/e6008652a7306c91155c9dd1b95a8e934a2ffefa-1920x1000.png)

通过对称距离计算（SDC，左），我们在将 xq 与之前量化的 xb 向量进行比较之前对其进行量化。ADC（右）跳过 xq 的量化，并将其直接与量化的 xb 向量进行比较。

要使用**index_factory**实现索引，我们可以编写：

In [ ]:
index = faiss.index_factory(d, "IVF256,PQ32x8")
index.train(xb)
index.add(xb)
D, I = index.search(xq, k)
recall(I)

由此，我们创建了包含256 个IVF 细胞的 IVFADC 索引；每个向量分别使用m和nbits值32和8通过 PQ 进行压缩。PQ默认使用nbits == 8所以我们也可以写"IVF256,PQ32"。

m : 原始向量被分割成的子向量的数量

nbits：每个子量化器使用的位数，我们可以计算每个子量化器使用的质心数量为 2**nbits

我们可以减少nbits以减少索引内存使用量，或增加 nbits 以提高召回率和搜索速度。然而，当前版本的 Faiss 确实将IVF ,PQ的nbits限制为>= 8。

还可以增加index.nprobe值来搜索更多 IVF 细胞 - 默认情况下，该值为1。



In [ ]:
index.nprobe = 8
D, I = index.search(xq, k)
recall(I)

这里我们有不同**nbits**和**nprobe**值的索引性能：

|      指数      | 探针 | 记起 | 搜索时间 | 记忆 |
| :------------: | :--: | :--: | :------: | :--: |
| IVF256，PQ32x4 |  1   | 27%  | 329微秒  | 25MB |
| IVF256，PQ32x4 |  6   | 45%  | 975微秒  | 25MB |
| IVF256，PQ32x8 |  1   | 30%  | 136微秒  | 40MB |
| IVF256，PQ32x8 |  8   | 74%  | 729微秒  | 40MB |

#### 优化产品量化

IVFADC和其他使用 PQ 的索引可以从优化产品量化( OPQ)中受益。

OPQ 的工作原理是旋转向量以使 PQ 中使用的子向量上的值分布变平。这对于数据分布不均匀的不平衡向量特别有利。

在 Faiss 中，我们添加 OPQ 作为预处理步骤。对于IVFADC，OPQ索引字符串看起来像“OPQ32，IVF256，PQ32”，其中OPQ32和PQ32中的32指的是PQ生成的代码中的字节数m 。  

Faiss中的OPQ矩阵并不是 整个 旋转和PQ过程。这只是旋转。下游必须包含 PQ 步骤才能实施 OPQ。

和以前一样，我们需要在初始化时训练索引。



In [ ]:
# we can add pre-processing vector rotation to
# improve distribution for the PQ step using OPQ
index = faiss.index_factory(d, "OPQ32,IVF256,PQ32x8")
index.train(xb)
index.add(xb)
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

Sift1M 数据集的数据分布已经很平衡，因此 OPQ 只给我们带来了召回性能的微小提升。当nprobe == 1时，我们的召回率从 30% 增加到 31%。

我们可以增加nprobe值来提高召回率（以速度为代价）。但是，由于我们向索引添加了预处理步骤，因此我们无法直接使用index.nprobe访问nprobe，因为该索引不再引用索引的 IVF 部分。

相反，我们必须在修改nprobe值之前提取IVF 索引- 我们可以使用extract_index_ivf函数来完成此操作。



In [ ]:
ivf = faiss.extract_index_ivf(index)
ivf.nprobe = 13
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

当**nprobe**值为**14 时**，我们返回 74% 的召回率。与单独 PQ 类似的召回结果，同时搜索时间从 729μs -> 1060μs 增加。

|      指数      | 探针 | 记起 | 搜索时间 |  记忆  |
| :------------: | :--: | :--: | :------: | :----: |
| IVF256，PQ32x4 |  1   | 30%  | 136微秒  | 40.2MB |
| IVF256，PQ32x4 |  1   | 31%  | 143微秒  | 40.3MB |
| IVF256，PQ32x8 |  8   | 74%  | 729微秒  | 40.2MB |
| IVF256，PQ32x8 |  13  | 74%  | 1060微秒 | 40.3MB |

我们将在本文后面看到 OPQ 可用于提高性能，但正如我们在这里看到的，情况并非*总是*如此。

![各种 nprobe 值的搜索时间（上）和召回率（下）。 我们纳入了“IVF256，Flat”进行比较。 扁平索引的内存使用量要高得多，为 520MB。](https://cdn.sanity.io/images/vr8gru94/production/2f8651fdfc87169f2b088c95d37a8a033701ad7a-1920x1080.png)

各种 nprobe 值的搜索时间（上）和召回率（下）。我们纳入了“IVF256，Flat”进行比较。扁平索引的内存使用量要高得多，为 520MB。

OPQ 还可用于降低预处理步骤中向量的维数。这个维度D必须是M的倍数，最好是D == 4M。为了将维度减少到64，我们可以使用"OPQ16_64,IVF256,PQ16"。



Multi-D-ADC 是指**多维**索引，以及在搜索时产生非对称距离计算的 PQ 步骤（如我们之前讨论的**）** [ **5** ] **。**

多**D-ADC**指数基于倒置多指数 (IMI)，是 IVF 的延伸。

IMI 在召回和搜索速度方面都优于 IVF，但确实增加了内存使用量 [7]。这使得 IMI 索引（例如多 D-ADC）在 IVFADC 无法完全达到所需的召回率和速度的情况下成为理想选择，并且您可以节省更多内存使用量。IMI 索引的工作方式与 IVF 非常相似，但 Voronoi 单元在向量维度上进行分割。它产生的结果类似于多级 Voronoi 单元结构。

![沃罗诺伊细胞分裂到多个向量子空间。 给定一个查询向量 xq，我们将每个 xq 子向量与其各自的子空间单元进行比较。](https://cdn.sanity.io/images/vr8gru94/production/ce81a066f489fe38b681d4fd7111f9cb5e4c804e-1920x1020.png)

沃罗诺伊细胞分裂到多个向量子空间。给定一个查询向量 xq，我们将每个 xq 子向量与其各自的子空间单元进行比较。

当我们使用 PQ 将矢量压缩添加到 IMI 时，我们会生成**多 D-ADC**索引。其中 ADC 是指将查询向量与 PQ 向量进行比较时进行的不对称距离计算。

将所有这些放在一起，我们可以使用索引工厂字符串“IMI2x8,PQ32”创建多 D-ADC 索引。



In [ ]:
index = faiss.index_factory(d, "IMI2x8,PQ32")
index.train(xb)  # index construction time is large for IMI
index.add(xb)

In [ ]:
imi = faiss.extract_index_ivf(index)  # access nprobe
imi.nprobe = 620
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

为了返回与 IVFADC 等效的类似召回，我们将搜索时间增加到 1.3 毫秒，这非常慢。但是，如果我们将 OPQ 添加到索引中，我们将返回更好的结果。



In [ ]:
index = faiss.index_factory(d, "OPQ32,IMI2x8,PQ32")  # lets try with OPQ
index.train(xb)  # index construction time is even larger for OPQ,IMI
index.add(xb)

In [ ]:
imi = faiss.extract_index_ivf(index)  # we increase nprobe
imi.nprobe = 100
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

对于 74% 的召回率，我们的 OPQ multi-D-ADC 索引速度最快，平均搜索时间仅为 461μs。

|       Index       | Recall | Search Time | Memory |
| :---------------: | :----: | :---------: | :----: |
|    IVF256,PQ32    |  74%   |    729µs    | 40.2MB |
|    IMI2x8,PQ32    |  72%   |   1350µs    | 40.8MB |
| OPQ32,IMI2x8,PQ32 |  74%   |    461µs    | 40.7MB |

和以前一样，我们可以使用nprobe微调索引以优先考虑召回或速度。

![各种 nprobe 值的搜索时间（上）和召回率（下）。 我们包含“IMI2x8，Flat”进行比较。 扁平索引的内存使用量要高得多，为 520MB。](https://cdn.sanity.io/images/vr8gru94/production/3500949c3274050f2c7bcc970dd74e24c46a7e95-1920x1080.png)

各种 nprobe 值的搜索时间（上）和召回率（下）。我们包含“IMI2x8，Flat”进行比较。扁平索引的内存使用量要高得多，为 520MB。

“**OPQ32，IMI2x8，PQ32**”是我们在低内存的召回率和速度方面最好的指标之一。然而，我们将看到我们可以通过以下索引进一步改进这些指标。



### HNSW Indexes

IVF 与 分层导航小型世界(HNSW) 图是我们最终的复合索引。该索引将我们的索引向量按照通常的 IVF 方式分割成单元格，但这次我们将使用 HNSW 优化该过程。

与我们之前的两个指数相比，采用 HNSW 的 IVF 可以产生相当或更好的速度以及显着更高的召回率，但代价是内存使用量更高*。*

在较高层面上，HNSW 基于*小世界图理论*，即网络中的所有顶点（*节点*）（无论有多大）都可以通过少量步骤遍历。

![](https://d33wubrfki0l68.cloudfront.net/2c46ea2a595c6ecb10a7c1d729bb85620bcc3b3e/b9423/images/composite-indexes-11.mp4)

可导航的小世界图的示例，图中的所有节点都通过少量的边遍历连接。小世界图论假设即使对于具有数十亿个顶点的巨大网络也是如此。

在这个小世界图中，我们看到了短程和长程链接。当遍历远程链接时，我们在图表中移动得更快。

HNSW 通过将图链接拆分为多个层来利用这一点。在较高的入口层，我们只找到远程链接。当我们向下移动层时，会添加较短距离的链接。

搜索时，我们从这些具有远程链接的较高层开始。这意味着我们的第一次遍历是跨越远程链接的。当我们向下移动层时，当我们遍历更多的短程链接时，我们的搜索变得更加精细。

![HNSW 图将包含远程和短程链接的典型图分解为多个层（层次结构）。 在搜索过程中，我们从最高层开始，它由远程链接组成。 当我们向下移动每一层时，链接变得更加细化。](https://cdn.sanity.io/images/vr8gru94/production/0b4b19e673df19f684dbfe5ed91f80f283cda10c-1920x1080.png)

HNSW 图将包含远程和短程链接的典型图分解为多个层（层次结构）。在搜索过程中，我们从最高层开始，它由远程链接组成。当我们向下移动每一层时，链接变得更加细化。

这种方法应该最大限度地减少遍历次数（加快搜索速度），同时仍然在较低层中执行非常精细的搜索（保持高召回率）。

那就是HNSW，但是我们如何才能将HNSW与IVF结合起来呢？

使用普通 IVF，我们引入查询向量并将其与每个细胞质心进行比较，识别最近的质心以限制我们的搜索范围。

为了将此过程与 HNSW 配对，我们生成所有这些细胞质心的 HNSW 图，使穷举质心搜索*近似*。

![HNSW 可用于使用 IVF 细胞质心快速找到近似最近邻。](https://cdn.sanity.io/images/vr8gru94/production/0c94d256ebe92cf3ffea7429f54ede89477a6513-1920x1080.png)

HNSW 可用于使用 IVF 细胞质心快速找到近似最近邻。

之前，我们一直使用具有 256 个细胞质心的 IVF 索引。256 的穷举搜索很快，并且没有理由使用具有如此少质心的近似搜索。

由于我们的单元格很少，因此每个单元格必须包含许多向量 - 仍将使用穷举搜索来搜索这些向量。在这种情况下，细胞质心上的 IVF+HNSW 没有帮助。

对于 IVF+HNSW 索引，我们需要将“少数质心和大单元”替换为“许多质心和小单元”。

对于我们的 1M 索引，建议nlist值为65536 [8]。但是，我们应该向index.train提供至少 30\*nlist == 1.97M个向量，但我们没有。所以16384或更小的nlist更合适。对于此数据集，nlist == 4096返回最高的召回率（速度较慢）。

使用 IVF+HNSW，我们使用 HNSW 快速识别近似最近的细胞质心，然后将我们的穷举搜索限制到那些最近的细胞。

可以使用"IVF4096_HNSW32,Flat"构建标准 IVF+HNSW 索引。利用这个，我们有：

- 4096 个IVF 单元。
- 单元质心存储在 HNSW 图中。每个质心都与其他32 个质心相连。
- 向量本身没有改变。它们是平面向量。

In [ ]:
index = faiss.index_factory(d, "IVF4096_HNSW32,Flat")
index.train(xb)
index.add(xb)

In [ ]:
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

In [ ]:
index.nprobe = 146
D, I = index.search(xq, k)
recall(I)

In [ ]:
%%timeit
index.search(xq, k)

有了这个索引，我们可以在 58.9μs -> 916μs 的搜索时间内实现从 25% -> 100% 召回率的令人难以置信的性能。

![各种 nprobe 值的搜索时间（上）和召回率（下）。 以更长的搜索时间为代价，我们可以通过减少 nlist 来提高召回率。](https://cdn.sanity.io/images/vr8gru94/production/ae39ade8751b67c4acdc7f960c9a0ee9e565ff95-1920x1080.png)

各种 nprobe 值的搜索时间（上）和召回率（下）。以更长的搜索时间为代价，我们可以通过减少 nlist 来提高召回率。

然而，IVF+HNSW 指数也并非没有缺陷。尽管我们拥有令人难以置信的召回率和快速的搜索速度，但这个索引的内存使用量是*巨大的*。我们的 1M 128 维向量产生的索引大小为 523MB+。

正如我们之前所做的那样，我们可以使用 PQ 和 OPQ 来减少这种情况，但这会减少召回率并增加搜索时间。



|             Index             | Recall | Search Time | Memory |
| :---------------------------: | :----: | :---------: | :----: |
|       IVF4096_HNSW,Flat       |  90%   |    550µs    | 523MB  |
|    IVF4096_HNSW,PQ32 (PQ)     |  69%   |    550µs    |  43MB  |
| OPQ32,IVF4096_HNSW,PQ32 (OPQ) |  74%   |    364µs    |  43MB  |

如果可以接受较低的召回率以最大限度地减少搜索时间和内存使用量，则带有 OPQ 的 IVF+HNSW 索引是理想的选择。另一方面，带有 PQ 的 IVF+HNSW 并没有比我们之前的*IVFADC*和*Multi-D-ADC*指数有任何优势。

|    Name     |   Index String    | Recall | Search Time | Memory |
| :---------: | :---------------: | :----: | :---------: | :----: |
|   IVFADC    |    IVF256,PQ32    |  74%   |    729µs    |  40MB  |
| Multi-D-ADC | OPQ32,IMI2x8,PQ32 |  74%   |    461µs    |  41MB  |



这就是本文的内容！我们介绍了复合索引以及如何使用 Faiss **index_factory**构建它们。我们研究了几个最流行的综合指数，包括：

- IVFADC
- Multi-D-ADC
- IVF-HNSW

通过索引和搜索 Sift1M 数据集，我们学习了如何修改每个索引的参数以优先考虑召回率、速度和内存使用情况。

根据我们在此介绍的内容，您将能够设计和测试各种复合索引，并更好地决定最适合您需求的索引结构。

# References
[1] Y.Chen, et al., Approximate Nearest Neighbor Search by Residual Vector Quantization (2010), Sensors

[2] Y. Matsui, et al., A Survey of Product Quantization (2018), ITE Trans. on MTA

[3] T. Ge, et. al., Optimized Product Quantization (2014), TPAMI

[4] H. Jégou, et al., Product quantization for nearest neighbor search (2010), TPAMI

[5] A. Babenko, V. Lempitsky, The Inverted Multi-Index (2012), CVPR

[6] H. Jégou, et al., Searching in One Billion Vectors: Re-rank with Source Coding (2011), ICASSP

[7] D. Baranchuk, et al., Revisiting the Inverted Indices for Billion-Scale Approximate Nearest Neighbors (2018), ECCV

[8] Guidelines to choose an index, Faiss wiki

[9] The Index Factory, Faiss wiki

